In [73]:
import xarray
import rioxarray

In [74]:
region = 'UCSB'
timestamp = '2024-02-10'

In [64]:
zarr_store = f'/tablespace/sentinel2/{region}_sharpend.zarr/'
ds = xarray.open_zarr(zarr_store)

In [66]:
ds['spatial_ref']

<xarray.DataArray 'spatial_ref' ()> Size: 8B
[1 values with dtype=int64]
Attributes: (12/18)
    GeoTransform:                      300000.0 20.0 0.0 4200000.0 0.0 -20.0
    crs_wkt:                           PROJCS["WGS 84 / UTM zone 11N",GEOGCS[...
    false_easting:                     500000.0
    false_northing:                    0.0
    geographic_crs_name:               WGS 84
    grid_mapping_name:                 transverse_mercator
    ...                                ...
    projected_crs_name:                WGS 84 / UTM zone 11N
    reference_ellipsoid_name:          WGS 84
    scale_factor_at_central_meridian:  0.9996
    semi_major_axis:                   6378137.0
    semi_minor_axis:                   6356752.314245179
    spatial_ref:                       PROJCS["WGS 84 / UTM zone 11N",GEOGCS[...

In [61]:
ds = ds.sel(time=timestamp).squeeze()
ds.rio.write_crs(32611, inplace=True)
ds.rio.set_spatial_dims('x', 'y', inplace=True)

<xarray.Dataset> Size: 90MB
Dimensions:               (y: 921, x: 1347, band: 10, detector: 7,
                           y_angles: 23, x_angles: 23)
Coordinates:
  * band                  (band) <U3 120B 'B5' 'B6' 'B7' ... 'B3' 'B4' 'B8'
  * detector              (detector) <U2 56B '10' '7' '9' '12' '8' '6' '11'
    spatial_ref           int64 8B 0
    time                  datetime64[ns] 8B 2024-02-10T18:45:21.024000
  * x                     (x) float64 11kB 3.146e+05 3.146e+05 ... 3.28e+05
  * x_angles              (x_angles) float64 184B 3e+05 3.05e+05 ... 4.1e+05
  * y                     (y) float64 7kB 4.171e+06 4.171e+06 ... 4.162e+06
  * y_angles              (y_angles) float64 184B 4.2e+06 4.195e+06 ... 4.09e+06
Data variables:
    AOT                   (y, x) float32 5MB dask.array<chunksize=(500, 500), meta=np.ndarray>
    CLD                   (y, x) float64 10MB dask.array<chunksize=(500, 500), meta=np.ndarray>
    SCL                   (y, x) float64 10MB dask.array<chunksize=(500, 500), meta=np.ndarray>
    SNW                   (y, x) float64 10MB dask.array<chunksize=(500, 500), meta=np.ndarray>
    WVP                   (y, x) float32 5MB dask.array<chunksize=(500, 500), meta=np.ndarray>
    reflectance           (y, x, band) float32 50MB dask.array<chunksize=(500, 500, 10), meta=np.ndarray>
    sun_azimuth_grid      (y_angles, x_angles) float32 2kB dask.array<chunksize=(23, 23), meta=np.ndarray>
    sun_zenith_grid       (y_angles, x_angles) float32 2kB dask.array<chunksize=(23, 23), meta=np.ndarray>
    viewing_azimuth_grid  (band, y_angles, x_angles, detector) float32 148kB dask.array<chunksize=(10, 23, 23, 7), meta=np.ndarray>
    viewing_zenith_grid   (band, y_angles, x_angles, detector) float32 148kB dask.array<chunksize=(10, 23, 23, 7), meta=np.ndarray>
Attributes: (12/84)
    AOT_QUANTIFICATION_VALUE:              1000.0
    AOT_QUANTIFICATION_VALUE_UNIT:         none
    AOT_RETRIEVAL_ACCURACY:                0.0
    AOT_RETRIEVAL_METHOD:                  CAMS
    BOA_QUANTIFICATION_VALUE:              10000
    BOA_QUANTIFICATION_VALUE_UNIT:         none
    ...                                    ...
    viewing_zenith_mean_B5:                5.57337094496921
    viewing_zenith_mean_B6:                5.61673664597282
    viewing_zenith_mean_B7:                5.66383856967182
    viewing_zenith_mean_B8:                5.44031658640393
    viewing_zenith_mean_B8A:               5.71403297081585
    viewing_zenith_mean_B9:                5.82625326117014

In [62]:
ds['reflectance'].to_dataset('band').rio.to_raster(f'{region}_{timestamp}_reflectances.tiff')

In [63]:
ds['sun_zenith_grid'] = ds['sun_zenith_grid'].interp(y_angles=ds.y, x_angles=ds.x, method='nearest').squeeze()
ds['sun_zenith_grid'].rio.to_raster(f'{region}_{timestamp}_sun_zenith.tiff')